In [1]:
#import des libraires nécessaires
import os
import numpy as np
import nibabel as nib
import pandas as pd
import seaborn as sns
import glob
import plotly.graph_objects as go
from scipy.stats import chi2_contingency
from importlib import reload
from itertools import combinations
%matplotlib inline
from IPython.display import display,Image
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [13]:
#Chemin 
masksLesionsonDTI=os.path.join("/NAS","coolio","vincent","data","chuStroke_2022","lesion_native")
synthSegOnFlair=os.path.join("/NAS","coolio","protocoles","cohorte_AVC_synthseg","2022")
leucopathiesWMHPOnFlair=os.path.join("/NAS","coolio","protocoles","cohorte_AVC_synthseg","WMHP","2022")

In [30]:
# Récupérer la liste des fichiers dans le répertoire masksLesionsonDTI
fichiers = glob.glob(os.path.join(masksLesionsonDTI, '*'))

# Créer le dictionnaire
dic_masks_dwi= {os.path.basename(fichier)[:-7].zfill(4): fichier for fichier in fichiers}
print(dic_masks_dwi)

{'262': '/NAS/coolio/vincent/data/chuStroke_2022/lesion_native/262.nii.gz', '370': '/NAS/coolio/vincent/data/chuStroke_2022/lesion_native/370.nii.gz', '321': '/NAS/coolio/vincent/data/chuStroke_2022/lesion_native/321.nii.gz', '168': '/NAS/coolio/vincent/data/chuStroke_2022/lesion_native/168.nii.gz', '141': '/NAS/coolio/vincent/data/chuStroke_2022/lesion_native/141.nii.gz', '243': '/NAS/coolio/vincent/data/chuStroke_2022/lesion_native/243.nii.gz', '010': '/NAS/coolio/vincent/data/chuStroke_2022/lesion_native/10.nii.gz', '205': '/NAS/coolio/vincent/data/chuStroke_2022/lesion_native/205.nii.gz', '107': '/NAS/coolio/vincent/data/chuStroke_2022/lesion_native/107.nii.gz', '093': '/NAS/coolio/vincent/data/chuStroke_2022/lesion_native/93.nii.gz', '341': '/NAS/coolio/vincent/data/chuStroke_2022/lesion_native/341.nii.gz', '193': '/NAS/coolio/vincent/data/chuStroke_2022/lesion_native/193.nii.gz', '001': '/NAS/coolio/vincent/data/chuStroke_2022/lesion_native/1.nii.gz', '150': '/NAS/coolio/vincent/

In [53]:
fichiers = glob.glob(os.path.join(leucopathiesWMHPOnFlair,'sub-*','ses-*','*WMHP.nii.gz'))

# Créer le dictionnaire
dic_leuco_mask = {'_'.join(os.path.basename(fichier).split('_')[1:3]): fichier for fichier in fichiers}
print(dic_leuco_mask)

{'0089_ses-2': '/NAS/coolio/protocoles/cohorte_AVC_synthseg/WMHP/2022/sub-22_0089/ses-2/sub-22_0089_ses-2_WMHP.nii.gz', '0089_ses-1': '/NAS/coolio/protocoles/cohorte_AVC_synthseg/WMHP/2022/sub-22_0089/ses-1/sub-22_0089_ses-1_WMHP.nii.gz', '0161_ses-2': '/NAS/coolio/protocoles/cohorte_AVC_synthseg/WMHP/2022/sub-22_0161/ses-2/sub-22_0161_ses-2_WMHP.nii.gz', '0161_ses-1': '/NAS/coolio/protocoles/cohorte_AVC_synthseg/WMHP/2022/sub-22_0161/ses-1/sub-22_0161_ses-1_WMHP.nii.gz', '0306_ses-2': '/NAS/coolio/protocoles/cohorte_AVC_synthseg/WMHP/2022/sub-22_0306/ses-2/sub-22_0306_ses-2_WMHP.nii.gz', '0306_ses-1': '/NAS/coolio/protocoles/cohorte_AVC_synthseg/WMHP/2022/sub-22_0306/ses-1/sub-22_0306_ses-1_WMHP.nii.gz', '0116_ses-2': '/NAS/coolio/protocoles/cohorte_AVC_synthseg/WMHP/2022/sub-22_0116/ses-2/sub-22_0116_ses-2_WMHP.nii.gz', '0116_ses-1': '/NAS/coolio/protocoles/cohorte_AVC_synthseg/WMHP/2022/sub-22_0116/ses-1/sub-22_0116_ses-1_WMHP.nii.gz', '0359_ses-2': '/NAS/coolio/protocoles/cohorte_A

In [57]:
fichiers = glob.glob(os.path.join(synthSegOnFlair,'sub*_resampled.nii.gz'))


# Créer le dictionnaire
dic_synthseg_mask = {'_'.join(os.path.basename(fichier).split('_')[1:3]): fichier for fichier in fichiers}
print(len(fichiers),len(dic_synthseg_mask))

849 803


In [55]:
len(dic_synthseg_mask)

803

In [56]:
len(dic_leuco_mask)

805

In [60]:
import nibabel as nib
from nilearn.image import resample_to_img

# Charger le fichier MGZ
mgz_file_path = '/NAS/tupac/romain/segMNIbyFS/mni/mri/aparc+aseg.mgz'
mgz_img = nib.load(mgz_file_path)

# Charger le fichier de référence NIfTI
ref_file_path = '/NAS/tupac/romain/segMNIbyVolbrain/mni_structures_MNI152_T1_1mm_resized.nii.gz'
ref_img = nib.load(ref_file_path)

# Rééchantillonner l'image MGZ pour correspondre à l'image de référence
resampled_img = resample_to_img(mgz_img, ref_img, interpolation='nearest')

# Enregistrer l'image rééchantillonnée en tant que fichier NIfTI
output_file_path = '/NAS/tupac/romain/segMNIbyFS/mni/mri/aparc+aseg_nearest_rl.nii.gz'
nib.save(resampled_img, output_file_path)

In [65]:
parts[1]

'wm_rh_S_temporal_transverse'

In [68]:
# Charger le fichier NIfTI
file_path = '/NAS/tupac/romain/segMNIbyFS/mni/mri/aparc+aseg_nearest_rl.nii.gz'
img = nib.load(file_path)

# Extraire les données de l'image
data = img.get_fdata()

# Obtenir les valeurs uniques
unique_labels = np.unique(data)
print(unique_labels)

# Chemin vers le fichier LUT
lut_file_path = '/NAS/coolio/protocoles/cohorte_AVC_synthseg/FreeSurferColorLUT.txt'

# Lire le fichier LUT
with open(lut_file_path, 'r') as file:
    lut_lines = file.readlines()

# Extraire les lignes correspondantes aux labels uniques
lut_dict = {}
for line in lut_lines:
    if line.startswith('#') or line.strip() == '':
        continue
    parts = line.split()
    if len(parts) > 1:
        label = int(parts[0])
        if label in unique_labels:
            lut_dict[label] = parts[1]

# Afficher les lignes correspondantes
for label in unique_labels:
    if label in lut_dict:
        print(f'{lut_dict[label]}')

[0.000e+00 2.000e+00 4.000e+00 5.000e+00 7.000e+00 8.000e+00 1.000e+01
 1.100e+01 1.200e+01 1.300e+01 1.400e+01 1.500e+01 1.600e+01 1.700e+01
 1.800e+01 2.400e+01 2.600e+01 2.800e+01 3.100e+01 4.100e+01 4.300e+01
 4.400e+01 4.600e+01 4.700e+01 4.900e+01 5.000e+01 5.100e+01 5.200e+01
 5.300e+01 5.400e+01 5.800e+01 6.000e+01 6.300e+01 7.700e+01 2.510e+02
 2.520e+02 2.530e+02 2.540e+02 2.550e+02 1.002e+03 1.003e+03 1.005e+03
 1.006e+03 1.007e+03 1.008e+03 1.009e+03 1.010e+03 1.011e+03 1.012e+03
 1.013e+03 1.014e+03 1.015e+03 1.016e+03 1.017e+03 1.018e+03 1.019e+03
 1.020e+03 1.021e+03 1.022e+03 1.023e+03 1.024e+03 1.025e+03 1.026e+03
 1.027e+03 1.028e+03 1.029e+03 1.030e+03 1.031e+03 1.034e+03 1.035e+03
 2.002e+03 2.003e+03 2.005e+03 2.006e+03 2.007e+03 2.008e+03 2.009e+03
 2.010e+03 2.011e+03 2.012e+03 2.013e+03 2.014e+03 2.015e+03 2.016e+03
 2.017e+03 2.018e+03 2.019e+03 2.020e+03 2.021e+03 2.022e+03 2.023e+03
 2.024e+03 2.025e+03 2.026e+03 2.027e+03 2.028e+03 2.029e+03 2.030e+03
 2.031

In [78]:
glob.glob(csv_files_path)[0]

'/NAS/coolio/protocoles/cohorte_AVC_synthseg/DWIinMNI/260_lesion_voxels4VB.csv'

In [86]:
with open(headers_file_path, 'r') as file:
    headers = file.read().strip().split('\n')
headers

['3rd ventricle',
 '4th ventricle',
 'Right accumbens',
 'Left accumbens',
 'Right amygdala',
 'Left amygdala',
 'Brainstem',
 'Right caudate',
 'Left caudate',
 'Right cerebellum exterior',
 'Left cerebellum exterior',
 'Right cerebellum White Matter',
 'Left cerebellum White Matter',
 'Right cerebral White Matter',
 'Left cerebral White Matter',
 'Right hippocampus',
 'Left hippocampus',
 'Right inf. lateral ventricle',
 'Left inf. lateral ventricle',
 'Right lateral ventricle',
 'Left lateral ventricle',
 'Right pallidum',
 'Left pallidum',
 'Right putamen',
 'Left putamen',
 'Right thalamus',
 'Left thalamus',
 'Right ventral DC',
 'Left ventral DC',
 'Lobules I-V',
 'Lobules VI-VII',
 'Lobules VIII-X',
 'Left basal forebrain',
 'Right basal forebrain',
 'Right anterior cingulate gyrus',
 'Left anterior cingulate gyrus',
 'Right anterior insula',
 'Left anterior insula',
 'Right anterior orbital gyrus',
 'Left anterior orbital gyrus',
 'Right angular gyrus',
 'Left angular gyrus',


In [92]:
glob.glob(csv_files_path)[0].split('/')[-1].split('_')[0]

'260'

# Aggregation des fichiers csv pour Vobrain

In [93]:
import glob
import pandas as pd

# Chemin vers les fichiers CSV
csv_files_path = '/NAS/coolio/protocoles/cohorte_AVC_synthseg/DWIinMNI/*_lesion_voxels4VB.csv'

# Chemin vers le fichier contenant les en-têtes des colonnes
headers_file_path = '/NAS/coolio/protocoles/cohorte_AVC_synthseg/VB_labels.txt'

# Lire les en-têtes des colonnes
with open(headers_file_path, 'r') as file:
    headers = file.read().strip().split('\n')

# Ajouter la colonne "SUBJ" au début des en-têtes
headers = ['SUBJ'] + headers

# Lire tous les fichiers CSV, ajouter la colonne "SUBJ" et les concaténer en un seul DataFrame
all_data = pd.concat([pd.read_csv(f, names=headers[1:]).assign(SUBJ=f.split('/')[-1].split('_')[0]) for f in glob.glob(csv_files_path)])

# Réorganiser les colonnes pour que "SUBJ" soit la première
all_data = all_data[headers]

# Enregistrer le DataFrame concaténé dans un fichier CSV unique
output_csv_path = '/NAS/coolio/protocoles/cohorte_AVC_synthseg/combined_lesion_voxels4VB.csv'
all_data.to_csv(output_csv_path, index=False)

# Aggregation des fichiers csv pour Aparc

In [94]:
import glob
import pandas as pd

# Chemin vers les fichiers CSV
csv_files_path = '/NAS/coolio/protocoles/cohorte_AVC_synthseg/DWIinMNI/*_lesion_voxels4APARC.csv'

# Chemin vers le fichier contenant les en-têtes des colonnes
headers_file_path = '/NAS/coolio/protocoles/cohorte_AVC_synthseg/APARC+ASEG_labels.txt'

# Lire les en-têtes des colonnes
with open(headers_file_path, 'r') as file:
    headers = file.read().strip().split('\n')

# Ajouter la colonne "SUBJ" au début des en-têtes
headers = ['SUBJ'] + headers

# Lire tous les fichiers CSV, ajouter la colonne "SUBJ" et les concaténer en un seul DataFrame
all_data = pd.concat([pd.read_csv(f, names=headers[1:]).assign(SUBJ=f.split('/')[-1].split('_')[0]) for f in glob.glob(csv_files_path)])

# Réorganiser les colonnes pour que "SUBJ" soit la première
all_data = all_data[headers]

# Enregistrer le DataFrame concaténé dans un fichier CSV unique
output_csv_path = '/NAS/coolio/protocoles/cohorte_AVC_synthseg/combined_lesion_voxels4APARC.csv'
all_data.to_csv(output_csv_path, index=False)

# Aggregation des fichiers csv pour JHU

In [96]:
import glob
import pandas as pd

# Chemin vers les fichiers CSV
csv_files_path = '/NAS/coolio/protocoles/cohorte_AVC_synthseg/DWIinMNI/*_lesion_voxels4JHU.csv'

# Chemin vers le fichier contenant les en-têtes des colonnes
headers_file_path = '/NAS/coolio/protocoles/cohorte_AVC_synthseg/JHU_labels.txt'

# Lire les en-têtes des colonnes
with open(headers_file_path, 'r') as file:
    headers = file.read().strip().split('\n')

# Ajouter la colonne "SUBJ" au début des en-têtes
headers = ['SUBJ'] + headers

# Lire tous les fichiers CSV, ajouter la colonne "SUBJ" et les concaténer en un seul DataFrame
all_data = pd.concat([pd.read_csv(f, names=headers[1:]).assign(SUBJ=f.split('/')[-1].split('_')[0]) for f in glob.glob(csv_files_path)])

# Réorganiser les colonnes pour que "SUBJ" soit la première
all_data = all_data[headers]

# Enregistrer le DataFrame concaténé dans un fichier CSV unique
output_csv_path = '/NAS/coolio/protocoles/cohorte_AVC_synthseg/combined_lesion_voxels4JHU.csv'
all_data.to_csv(output_csv_path, index=False)